In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyuvdata import UVData
%matplotlib notebook

# HERA Data Analysis Part I: Reading, Manipulating, and Visualizing Visibility Data  <a class="tocSkip">

### *Workshop Leaders: Bobby Pascua, Josh Dillon, and James Aguirre*  <a class="tocSkip">
#### (Originally developed by Carina Cheng & Josh Dillon)  <a class="tocSkip">

# Reading Files and Metadata

In this section, we'll go over how to read HERA data and some basic features of the `UVData` class that we will load the data into. For a detailed reference, you should browse [the pyuvdata documentation](https://pyuvdata.readthedocs.io/en/latest/). The examples covered here will use data from a small file prepared by Josh Dillon: `zen.2458098.40887.HH.calibrated_downselected.uvh5`.

In [ ]:
sample_file = "zen.2458098.40887.HH.calibrated_downselected.uvh5"
uvdata = UVData()  # Initialize the UVData object
uvdata.read(sample_file)  # Then read in the data

This function loads in the metadata of the the data file when it's constructed. So for example, we can answer some questions:

* How many antennas are in the array and what are their antenna numbers? 

* How many frequency channels are there in the array. What range do they cover in MHz?

* How many time integrations are there in the file?


In [ ]:
n_ants = uvdata.Nants_data  # Number of antennas with data associated with them.
n_ants_tot = uvdata.Nants_telescope  # Number of antennas in the array.
all_ants = uvdata.antenna_numbers
data_ants = set(uvdata.ant_1_array).union(uvdata.ant_2_array)

print(f"There are {n_ants_tot} antennas in the array, and they are:\n{all_ants}\n")
print(f"There are {n_ants} antennas with data, and they are:\n{data_ants}\n")

freqs_MHz = np.unique(uvdata.freq_array) / 1e6
fmin = freqs_MHz.min()
fmax = freqs_MHz.max()
print(f"There are {uvdata.Nfreqs} frequencies, covering {fmin:.2f} MHz to {fmax:.2f} MHz.\n")
print(f"There are {uvdata.Ntimes} integrations.")

## Exercise 1 

Now it's your turn. Try to answer these basic questions about the data:

1. What is the width of each frequency channel in MHz?

2. What range of Julian Dates do they span?

3. What is the lengh of each time integration in seconds?

4. What range of LSTs (local sidreal times) do the data span?

5. How many baselines are in the file? How many should there be? Why don't they agree? Hint: check the `history` attribute of the `UVData` object.

In [ ]:
# your answers here (hint: look at the attributes of the UVData object)


## Exercise 2 

Now let's plot our array to look at where the antennas are. (Hint, see what the `get_ENU_antpos` method does.)

In [ ]:
# Fill in the gaps in the code below

# Get the antenna positions and numbers

plt.figure()
plt.scatter(, , marker='.', color='k', s=3000)  # plot the antenna positions with black circles
for i, ant in enumerate():  # loop over antennas
    plt.text(, , , color='w', va='center', ha='center')  # label antenna numbers
plt.xlabel()
plt.ylabel()
plt.axis('equal');

# Retrieving Data and Flags

`UVData` objects provide a simple interface for obtaining data and flags. In order to retrieve data and/or flags, we need to know the baselines and polarizations for which data is available. We can investigate this using the `get_antpairpols` method:

In [ ]:
print(uvdata.get_antpairpols())

In [ ]:
# Let's get the flags and data for a long East-West baseline
antpairpol = (65, 71, 'xx')
data = uvdata.get_data(antpairpol)
flags = uvdata.get_flags(antpairpol)

Let's take a moment to inspect the data and flags. The data and flags are stored as `numpy` arrays. What are the array shapes and what data types are used? What do you think these correspond to?

In [ ]:
# Your code here


something about visualizing data after applying flags

In [ ]:
# We'll be applying flags to different data products frequently,
# so let's make a helper function to do it
def apply_flags(data, flags):
    return np.where(flags, np.nan+1j*np.nan, data)

In [ ]:
modified_data = apply_flags(data, flags)

In [ ]:
integration = 30 # this is the integration index, 30 out of 60 total integrations
plt.figure()
plt.plot(freqs_MHz, modified_data[integration].real, 'b-', label='Real part')
plt.plot(freqs_MHz, modified_data[integration].imag, 'r-', label='Imag part')
plt.xlabel('Frequency (MHz)')
plt.ylabel('Visibility (Jy)')
plt.grid(); plt.legend();

## Exercise 3
Plot visibilities, with bad data removed, as a function of LST for a few different frequency channels. Label each channel's frequency and include a legend.

In [ ]:
chans = [100,300,750]

# your answer here

# Waterfall Plots

## Exercise 4

(in breakout groups)

Recall that

$\LARGE{V_{ij}(\nu) = \int B_{ij}(\mathbf{\hat{r}}, \nu) I(\mathbf{\hat{r}},\nu) e^{-(2 \pi i \nu / c)(\mathbf{b_{ij}}\cdot \mathbf{\hat{r}})}} d\Omega$.


For simplicity, imagine that we have a two-element interferometer sitting on the equator of the earth with the two antennas separated by a vector that is perfectly aligned with the east-west direction. Make a sketch of what you think the phase of a visibility should look like, as a function of time if the sky contains a single source is located at zero-declination (is transiting through the zenith of the celestial sphere). 

Make another sketch of how the phase should evolve if the pair of antennas are aligned perfectly in the north-south direction. 

How would you expect the phase evolution of a longe EW baseline to compare to the phase evolution of a short EW baseline?

**Now Make a Waterfall**

Another useful way of visualizing our data is to plot it as a waterfall plot. A waterfall plot is a two-dimensional plot of the visibility (the cross-correlated signal between a pair of antennas) as a function of time (y-axis) and frequency (x-axis). We can use the `plt.imshow` to plot the amplitude and phase of the same baseline as above. Note that the keyword `extent` takes in 4 arguments which define the plot axes extent in the order of (xmin, xmax, ymin, ymax), and we've massaged our axes to display frequencies in MHz and times in LST hours.

In [ ]:
# Let's use a logarithmic colormap for visualizing the visibility amplitude
vmin = np.abs(data[~flags]).min()  # Get the smallest non-flagged visibility amplitude
vmax = np.abs(data[~flags]).max()  # and the largest
norm = plt.cm.colors.LogNorm(vmin=vmin, vmax=vmax)  # This will tell the plotter how to format the colorbar
lsts_hr = lsts * 12 / np.pi

# The extent of a plot is [left, right, bottom, top]
# Remember that imshow has the vertical axis increasing downward!
extent = [freqs_MHz[0], freqs_MHz[-1], lsts_hr[-1], lsts_hr[0]]

# Plot absolute value of visibility
plt.figure(figsize=(8,4))
plt.subplot(121)  # Use 1 row and 2 columns of plots; select the left plot
plt.imshow(np.abs(data), aspect='auto', norm=norm, interpolation='none', extent=extent)
plt.colorbar(label='Visibility Amplitude (Jy)')
plt.xlabel('Frequency (MHz)')
plt.ylabel('LST (hours)')
plt.title('Amplitude')

# Plot phase of visibility
plt.subplot(122)  # Now select the right plot
plt.imshow(np.angle(data), aspect='auto', cmap='twilight', interpolation='none', extent=extent)
plt.colorbar(label='Visibility Phase (Radians)')
plt.xlabel('Frequency (MHz)')
plt.ylabel('LST (hours)')
plt.title('Phase')

plt.tight_layout();

Some features to note from the waterfall plots:
* RFI
* Band edges
* Frequency and temporal structure

Thankfully, someone has already done a bunch of work to figure out which data might be bad and flagged it. Here is what the flags look like:

In [ ]:
# Plot RFI Flags
plt.figure(figsize=(8,4))
plt.subplot(121)
plt.imshow(flags.astype(float), aspect='auto', interpolation='none', extent=extent)
plt.colorbar()
plt.xlabel('Frequency (MHz)')
plt.ylabel('LST (hours)')
plt.title('Flags')

# Plot phase of visibility
plt.subplot(122)
plt.imshow(np.angle(modified_data), aspect='auto', cmap='twilight', interpolation='none', extent=extent)
plt.colorbar(label='Visibility Phase (Radians)')
plt.xlabel('Frequency (MHz)')
plt.ylabel('LST (hours)')
plt.title('Flagged Phase')

plt.tight_layout();

## Exercise 5
Exercise: Now try a short baseline and a long baseline of the same orientation. Make waterfall plots (in both amplitude and phase) for both. How do they differ?

In [ ]:
short_ew_bl = (37, 38, 'xx')
long_ew_bl = (65, 71, 'xx')

# your answer here
plt.figure(figsize=(10,5))

plt.subplot(221)


plt.subplot(222)


plt.subplot(223)


plt.subplot(224)


plt.tight_layout();

## Exercise 6
Now try comparing a N/S baseline to a E/W one of similar lengths do the waterfall plots differ for an E/W baseline vs. a N/S baseline? Does their difference make sense? 

In [ ]:
ew_bl = (37, 38, 'xx')
ns_bl = (51, 83, 'xx')

# your answer here
plt.figure(figsize=(10,8))

plt.subplot(221)


plt.subplot(222)


plt.subplot(223)


plt.subplot(224)


plt.tight_layout();

# The Delay Transform

The delay transform is a clever technique we use to isolate bright foregrounds in our data (which we can then filter out). The delay transform is simply the Fourier transform of the visibility along frequency. 


## Exercise 7

(breakout groups)

At a fixed time, our data should have two contributions: smooth foregrounds (that occupy small delays) and noise (that occupies many delays). As a group, make a sketch of what you would expect the frequency-to-delay Fourier transform (the delay transform) of this data to look like a single time.


## Exercise 8

Try implementing the delay transform using `np.fft.fft` by following the steps below.

In [ ]:
bl = (51,83,'xx')
vis = uvdata.get_data(bl)

# 1) Fourier transform "vis" along the frequency axis (don't forget to fftshift after)
vis_dt = # your answer here 

# 2) Find the frequency width of a channel in GHz
freq_width = # your answer here

#3) Convert frequencies to delays. Numpy's fftfreq function takes two arguments: 
#   the number of frequencies, and the frequeny width you calculated above
delays = # your answer here

# Setup the normalization and extent for the waterfall
vmin = np.abs(vis_dt).min()
vmax = np.abs(vis_dt).max()
norm = plt.cm.colors.LogNorm(vmin, vmax)
new_extent = (delays[0], delays[-1], lsts_hr[-1], lsts_hr[0])

plt.figure()
plt.imshow(np.abs(vis_dt), aspect='auto', interpolation='none', norm=norm, extent=new_extent)
plt.colorbar(label='Visibility Amplitude (Jy)')
plt.xlabel('Delay (ns)')
plt.ylabel('LST (hours)')
plt.title('Delay Transform');

## Exercise 9


Does this waterfall match your expectations? 

The delay transform looks a lot like a sinusoid. 

What is the Fourier tranform of a sinusoid? 

Can you identify this source of this sinusoid in the time-frequency waterfall of the data? Hint: use a linear colorscale ranging from about 0 to 1000. 

What do you think causes these features?

In [ ]:
# your answer here


**Solution:**

Recall that the Fourier transform of a sinusoid is a delta-function. Thus, if we look at the time-**frequency** waterfall, we should expect to see some delta-function like features. Indeed, we do see a number of such spiky prominences in the above plot. 

These spikes are narrowband Radio Frequency Interference (RFI)&mdash;things like FM radio stations and satellite communications signals.

## Exercise 10

If we set the values of the channels where there is RFI to zero, what do you think the delay-transformed data would look like?

## Exercise 11

Now plot the delay transform again, but this time set the flagged channels to 0 before taking the delay transform.

In [ ]:
# your answer here


That's better. So what do we see here? All the bright stuff at low delay values correspond to bright foreground sources that are smooth in frequency (and therefore "peaky" in delay). This is nice for us because we've isolated the foregrounds and we can filter them out easily in this space.

Let's think about what "delay" means physically. A delay can be thought of as the time difference between when a lightwave hits one antenna and when it hits the second antenna. In other words, there is a time lag between light hitting each antenna depending on the direction it comes from and the orientation of the baseline.

Where would the light have to be coming from for the time delay to be zero? Where would it need to be coming from to produce a maximum delay (hint: we call this the horizon limit)?

## Exercise 12
For the baseline we picked earlier, calculate the theoretical maximum delay (in nanoseconds), using $t = d/c$, where $t$ is the time delay, $d$ is the baseline distance, and $c$ is the speed of light. You can calculate the baseline distance using the saved `antpos` and `ants` variables from earlier, or approximate it using the antenna layout plot.

In [ ]:
# your answer here


A cool trick for a faster calculation is to convert your distance $d$ to feet. That number is also approximately the time delay in nanoseconds! 

Also note that the foregrounds in our delay-transform plot spread out past this horizon limit. This is due to all sorts of effects, but the biggest one is the RFI gaps. However, we have algorithms that can deal with these in a smarter way than just setting them to 0. 

# Delay-Filtering&mdash;Interpolating the RFI gaps

While our waterfall in exercise 11 is certainly an improvement over leaving the RFI spikes in, the ringing structures that are caused by RFI gaps are only roughly a factor of 10-100 times lower than the foreground amplitude and cover all delays. We can think of these features as being an extra-nasty window function that corresponds to a top-hat window but now with gaps in it. This window function convolves our foregrounds and smears them to all delays. 

The 21-cm signal is ~$10^4$ smaller than the foregrounds, so the 1-10% flagging side-lobes are still a huge problem. 

One way to fix this is to use a delay <b>CLEAN</b>ing. Delay <b>CLEAN</b> is an algorithm that fits the data to complex sinusoids (with the same wavelengths as the wavelengths that we expect the foregrounds to have) and uses these sinusoids to interpolate foreground values in the flagged RFI channels. By performing this in-painting, we can eliminate the flagging side-lobes from exercise 11. 


We call the collection of sinusoids used to interpolate our data **CLEAN components** or our **CLEAN model**. Data that has been interpolated with these sinusoids is **CLEANed data**. 

Since we don't actually care about the foregrounds, we can also just subtract our fitted CLEAN components from the data. 

Here is how you can apply the delay-filtering code used by HERA to your data. 

In [ ]:
from IPython.display import Image
Image('smooth_power_spectrum_gaps_clean.png')

In [ ]:
from hera_cal import delay_filter
from hera_cal.io import HERAData
import matplotlib
hd = HERAData(sample_file)
data, flags, nsamples = hd.read()

bl = (51,83,'xx')
df = delay_filter.DelayFilter(hd) # establish delay filter object
df.run_filter(to_filter=[bl]) # filter the specific key we want (otherwise it takes a long time to do all keys)

clean_data = df.clean_data[bl] # filtered visibility (i.e. the cleaned visibility)
clean = df.clean_model[bl] # low delay modes we don't want (i.e. the stuff that gets filtered out)
resid = df.clean_resid[bl] # low delay modes we don't want (i.e. the stuff that gets filtered out)

# Plot
plt.figure(figsize=(10,4))
plt.subplot(141)
plt.imshow(np.abs(data[bl] / ~flags[bl]), aspect='auto', norm=matplotlib.colors.LogNorm(1e0,1e2), interpolation='none',
          extent=(hd.freqs[0]/1e6, hd.freqs[-1]/1e6, 
                  hd.lsts[-1]*12/np.pi, hd.lsts[0]*12/np.pi))
plt.colorbar(label='Visibility Amplitude (Jy)')
plt.xlabel('Frequency (MHz)')
plt.ylabel('LST (hours)')
plt.title('Original Visibility')

plt.subplot(142)
plt.imshow(np.abs(clean_data), aspect='auto', norm=matplotlib.colors.LogNorm(1e0,1e2), interpolation='none',
          extent=(hd.freqs[0]/1e6, hd.freqs[-1]/1e6, 
                  hd.lsts[-1]*12/np.pi, hd.lsts[0]*12/np.pi))
plt.colorbar(label='Visibility Amplitude (Jy)')
plt.xlabel('Frequency (MHz)')
plt.ylabel('LST (hours)')
plt.title('Cleaned Visibility')

plt.subplot(143)
plt.imshow(np.abs(clean), aspect='auto', norm=matplotlib.colors.LogNorm(1e0,1e2), interpolation='none',
          extent=(hd.freqs[0]/1e6, hd.freqs[-1]/1e6, 
                  hd.lsts[-1]*12/np.pi, hd.lsts[0]*12/np.pi))
plt.colorbar(label='Visibility Phase (Radians)')
plt.xlabel('Frequency (MHz)')
plt.ylabel('LST (hours)')
plt.title('CLEAN Components')


plt.subplot(144)
plt.imshow(np.abs(resid), aspect='auto', norm=matplotlib.colors.LogNorm(1e0,1e2), interpolation='none',
          extent=(hd.freqs[0]/1e6, hd.freqs[-1]/1e6, 
                  hd.lsts[-1]*12/np.pi, hd.lsts[0]*12/np.pi))
plt.colorbar(label='Visibility Phase (Radians)')
plt.xlabel('Frequency (MHz)')
plt.ylabel('LST (hours)')
plt.title('CLEAN Residual')

plt.tight_layout();

Comparing the left and middle left plots, we see that we've interpolated over the RFI gaps with smooth CLEAN components (middle right plot). Subtracting the CLEAN components from the original data reduces the amplitude of our visibility by a couple orders of magnitude (right plot)! We also see that some of the foreground structure we see in the left plot has been removed, as the right plot looks more noise-like. Delay-filtering is a powerful tool for HERA; afterall, we are trying to find a tiny signal (the 21cm EoR signal) buried underneath a lot of stuff we don't care about!

## Exercise 13
Try running the delay filter with `to_filter=[bl]` and `min_dly=6000` to use the CLEAN part of the filter without the "filtering" part. Now make a waterfall of `df.clean_data[bl]` in delay vs. LST. How does this compare to what you did above?

In [ ]:
bl = (65,71,'xx')

# your answer here
